In [50]:
# Initialize Otter
import otter
grader = otter.Notebook("ps4.ipynb")

### Question 1
In this question you will implement a class called `Matrix`. A matrix will be modeled as a collection of vectors all of the same dimension, and you may use the `Vector` class that we created in Lecture 5 (included with this problem set as `vector.py`) to implement it:

In [51]:
# from vector import Vector
# class Matrix():
#     def __init__(self, *args):
#         # print(args)
#         # No args
#         if not args:
#             # print("No Data")
#             # self.data = None
#             raise ValueError("Matrix should require at least one vector for initialization")
#         else:
#             self.dimension = -1
#             self.data = []
#             for i, arg in enumerate(args):
#                 # Check Type
#                 if not isinstance(arg, Vector):
#                     raise TypeError("Input arguments should be vectors")
#                 # Check dimensnion
#                 if i==0:
#                     self.dimension = len(arg)
#                 else:
#                     if self.dimension!=len(arg):
#                         raise ValueError("Vectors should all have the same dimension")
#                 # Add arg to data 
#                 self.data.append(arg)
#         self._val = tuple(list(self.data))
#         print(self._val)


from vector import Vector
from numbers import Number

class Matrix():
    def __init__(self, *vectors):
        #check if there are arguments passed
        if not vectors:
            raise ValueError("Matrix should require at least one vector for initialization")
        data = []
        dimension = -1
        for index in range(len(vectors)):
            #check if it's Vector
            if not isinstance(vectors[index],Vector):
                raise TypeError("Input arguments should be vectors")
            if index == 0:
                dimension = len(vectors[index])
            elif not len(vectors[index]) == dimension:
                raise ValueError("Vectors should all have the same dimension")

            data.append(vectors[index])
        
        self.data = tuple(data)

    @property
    def rows(self):
        return len(self.data[0])
    
    @property
    def cols(self):
        return len(self.data)

    def __str__(self):
        result = ''
        for row in range(self.rows):
            row_string = '('
            for col in range(self.cols):
                row_string += (str(self.data[col][row]))
                if (col != self.cols - 1):
                    row_string += ', '
            row_string += ')'
            result += row_string
            if row != self.rows - 1:
                result += '\n'
        return result

    @property
    def dims(self):
        return (self.rows, self.cols)

    def __iter__(self):
        for index in range(len(self.data)):
            yield self.data[index]
    
    def __add__(self, otherMatrix):
        if not self.dims == otherMatrix.dims:
            raise ValueError('Cannot add matrices of different dimensions')

        data = []
        for index in range(len(self.data)):
            data.append(self.data[index] + otherMatrix.data[index])
        return Matrix(*data)

    def __mul__(self, number):
        if not isinstance(number, Number):
            raise TypeError('...')

        data = []
        for index in range(len(self.data)):
            data.append(self.data[index] * number)
        return Matrix(*data)
    
    def __rmul__(self, number):
        return self.__mul__(number)

    def __getitem__(self, position):
        row = position[0]
        col = position[1]
        if row < 0 or row > self.rows - 1:
            raise IndexError('...')
        if col < 0 or col > self.cols - 1:
            raise IndexError('...')

        return self.data[col][row]

    def transpose(self):
        data = []
        for row in range(self.rows):
            row_elements = []
            for col in range(self.cols):
                row_elements.append(self.data[col][row])
            data.append(Vector(*row_elements))
        return Matrix(*data)

    def __matmul__(self, otherMatrix):
        if not self.dims[1] == otherMatrix.dims[0]:
            raise ValueError('Incompatible matrix dimensions')

        data = []
        for col in range(otherMatrix.cols):
            col_elements = []
            for row in range(self.rows):
                col_element = 0
                for k in range(self.cols):
                    col_element += self.data[k][row] * otherMatrix.data[col][k]
                col_elements.append(col_element)
            data.append(Vector(*col_elements))
        
        return Matrix(*data)

    def __eq__(self, otherMatrix):
        if isinstance(otherMatrix, Number):
            return False
        if not self.dims == otherMatrix.dims:
            return False

        isEqual = True
        for row in range(self.rows):
            for col in range(self.cols):
                if self.data[col][row] != otherMatrix.data[col][row]:
                    isEqual = False
        
        return isEqual


                

**1(a)** (1 pt) The constructor of `Matrix` should accept one or more `Vector` objects, which will represent the *columns* of the matrix.

```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(5, 6, 7)
>>> v3 = Vector(8, 9, 10)
>>> Matrix(v1)
<__main__.Matrix at 0x10efbabb0>
>>> Matrix(v1, v2)
<__main__.Matrix at 0x117023640>
>>> Matrix(v1, v2, v3)
<__main__.Matrix at 0x117023040>
```

Additionally, it should perform the following checks:

1.  It should verify that each argument is of the correct type:
    ```
    >>> Matrix(1, "", v1)
    TypeError: Input arguments should be vectors
    ```
    
2.  It should verify that each argument has the same dimension.

    ```
    >>> v1 = Vector(1, 2, 3)
    >>> v2 = Vector(5, 6)
    >>> Matrix(v1, v2)
    ValueError: Vectors should all have the same dimension
    ```

In [52]:
grader.check("1a")

1a results: All test cases passed!

**1(b)** (2 pts) Calling `str` on a `Matrix` object should produce a nice looking text representation:

```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> v3 = Vector(7, 8, 9)
>>> M = Matrix(v1, v2, v3)
>>> print(M)
(1, 4, 7)
(2, 5, 8)
(3, 6, 9)
```

In [53]:
grader.check("q1b")

q1b results: All test cases passed!

**1(c)** (1 pt.) A matrix should have a attribute `dims` which returns a tuple of its dimensions (number of rows and columns):
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> M.dims
(3, 2)
```

In [54]:
grader.check("q1c")

q1c results: All test cases passed!

**1(d)** (1 pt.) A matrix should be iterable. Iterating over the matrix should return each of its columns as vectors.
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> for col in M: print(col)
(1, 2, 3)
(4, 5, 6)
```

In [55]:
grader.check("q1d")

q1d results: All test cases passed!

**1(e)** (2 pts) Two matrices of the same dimensions can be added together. 
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> print(M + M)
(2, 8)
(4, 10)
(6, 12)
```
However, matrices of different dimensions cannot be added together:
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> Matrix(v1) + Matrix(v1, v2)
ValueError: Cannot add matrices of different dimensions
```

In [56]:
grader.check("q1e")

q1e results: All test cases passed!

**1(f)** (2 pts) Matrices can be multipled by a numerical constant:
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> print(M * 2.5)
(2.5, 10.0)
(5.0, 12.5)
(7.5, 15.0)
>>> print("a" * M)
TypeError: ...
```

In [57]:
grader.check("q1f")

q1f results: All test cases passed!

**1(g)** (2 pts) Individual entries of a matrix can be accessed using the notation `M[i,j]`:
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> print(M)
>>> M[0, 0]
1
>>> M[2, 1]
6
>>> M[3, 3]
IndexError: ...
```

In [58]:
grader.check("q1g")

q1g results: All test cases passed!

**1(h)** (3 pts) A matrix can be transposed:
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> print(M)
(1, 4)
(2, 5)
(3, 6)
>>> print(M.transpose())
(1, 2, 3)
(4, 5, 6)
```

In [59]:
grader.check("q1h")

q1h results: All test cases passed!

**1(i)** (2 pts) Two matrices of conformable dimensions can be multiplied using the matrix multiplication operator (`@`):
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> print(M @ M.transpose())
(17, 22, 27)
(22, 29, 36)
(27, 36, 45)
>>> M @ M
ValueError: Incompatible matrix dimensions
```

In [60]:
grader.check("q1i")

q1i results: All test cases passed!

**1(j)** (2 pts) Matrices can be tested for equality:
```
>>> v1 = Vector(1, 2, 3)
>>> v2 = Vector(4, 5, 6)
>>> M = Matrix(v1, v2)
>>> M == M
True
>>> M == Matrix(v1)
False
>>> M == 1.0
False
>>> M == "matrix"
False
```

In [61]:
grader.check("q1j")

q1j results: All test cases passed!

### Question 2
[Connect Four](https://en.wikipedia.org/wiki/Connect_Four) is a two-player game where players drop colored discs vertically into a $6\times 7$ grid, with the object of being the first to form a horizontal, vertical, or diagonal line of four of one's own discs. If you've never played before and want to understand how the game works, [here is a free online version](https://www.cbc.ca/kids/games/play/connect-4).

In this question, you will implement Connect Four as a Python class. Your implementation will be slightly more general: it will allow the user to specify the numbers of rows and columns of the grid, as well as the number $k$ of markers in a row that you need to win. So, traditional Connect Four is `ConnectK(rows=6, cols=7, k=4)` in the class shown below.

In [62]:
class ConnectK:

    '''
    The game Connect K.
    
    Args:
        rows, cols: the dimensions of the grid. defaults to (6, 7) if not specified.
        k: the number of markers in a row a player needs in order to win, 
           defaults to 4 if not specified.
        
    Raises:
        ValueError if rows, cols and k are not all positive integers.
    '''
    
    def __init__(self, rows=6, cols=7, k=4):
        if rows <= 0 or cols <= 0 or k <= 0:
            raise ValueError('invalid rows, cols and/or k')

        self._rows = rows
        self._cols = cols
        self._k = k

        self._data = [['-' for col in range(cols)] for row in range(rows)]
        self.player = 0

    @property
    def rows(self):
        'the number of rows in the grid'
        return self._rows

    @property
    def cols(self):
        'the number of columns in the grid'
        return self._cols

    @property
    def k(self):
        'the number of markers in a row needed to win'
        return self._k

    @property
    def possible_moves(self):
        'a list of the currently possible moves (open columns)'
        result = []
        for col in range(self.cols):
            if self._data[0][col] == '-':
                result.append(col)
        return result
        
    def move(self, i):
        '''
        The player whose turn it is drops their disc in column i.
        
        Args:
            i: The 0-indexed column into which the player drop their disc.
            
        Returns:
            If the move results in the player winning the game, this function returns 0/1 representing the winner.
            Otherwise, it returns None.
            
        Raises:
            - ValueError if the move is not valid (column is full, or a non-existent column was specified).
        '''

        if not isinstance(i, Number):
            raise ValueError('...')

        row = 0
        while (row != self.rows and self._data[row][i] == '-'):
            row += 1

        if not row == 0:
            self._data[row - 1][i] = str(self.player)
        else:
            raise ValueError('...')

        # judge if player wins
        row = row - 1
        col = i

        count = 0
        while row - count >= 0 and self._data[row - count][col] == self._data[row][col]:
            count += 1
        up = count - 1

        count = 0
        while row + count <= self.rows - 1 and self._data[row + count][col] == self._data[row][col]:
            count += 1
        down = count - 1

        count = 0
        while col - count >= 0 and self._data[row][col - count] == self._data[row][col]:
            count += 1
        left = count - 1

        count = 0
        while col + count <= self.cols - 1 and self._data[row][col + count] == self._data[row][col]:
            count += 1
        right = count - 1

        count = 0
        while row - count >= 0 and col + count <= self.cols - 1 and self._data[row - count][col + count] == self._data[row][col]:
            count += 1
        right_up = count - 1

        count = 0
        while row + count <= self.rows - 1 and col + count <= self.cols - 1 and self._data[row + count][col + count] == self._data[row][col]:
            count += 1
        right_down = count - 1

        count = 0
        while row - count >= 0 and col - count >= 0 and self._data[row - count][col - count] == self._data[row][col]:
            count += 1
        left_up = count - 1

        count = 0
        while row + count <= self.rows - 1 and col - count >= 0 and self._data[row + count][col - count] == self._data[row][col]:
            count += 1
        left_down = count - 1

        if up + down + 1 >= self.k or left + right + 1 >= self.k or left_up + right_down + 1 >= self.k or left_down + right_up + 1 >= self.k:
            winner = self.player
        else:
            winner = None

        if self.player == 0:
            self.player = 1
        else:
            self.player = 0
        
        return winner
        
    def __repr__(self):
        return 'ConnectK(rows=' + str(self.rows) + ', cols=' + str(self.cols) + ', k=' + str(self.k) + ')'

    def __str__(self):
        'Return a string representation of the board'
        board = ''
        for row in range(self.rows):
            for col in range(self.cols):
                board += self._data[row][col]
            if row != self.rows - 1:
                board += '\n'
        return board

**2(a)** (2 pts) The constructor should accept three arguments, `rows`, `columns`, and `k`, and validate them:
```
>>> ConnectK(1, 2, 3)
ConnectK(rows=1, cols=2, k=3)
>>> ConnectK(0, 2, 3)
ValueError: invalid rows, cols and/or k
```
The default arguments for rows, columns, and k should be 6, 7 and 4, respectively:
```
>>> ConnectK()
ConnectK(rows=6, cols=7, k=4)
```
The values of each attribute should be accessible as properties:
```
>>> c = ConnectK(5, 6, 7)
>>> c.rows
5
>>> c.cols
6
>>> c.k
7
```

In [63]:
grader.check("q2a")

q2a results: All test cases passed!

**2(b)** (3 pts) Converting a `ConnectK` game to a string should return a text representation of the grid. The first player is labeled `0`, the second player is labeled `1`, and empty grid squares are shown as a dash `-`.
```
>>> c = ConnectK()
>>> print(c)
-------
-------
-------
-------
-------
-------
>>> c.move(0)
>>> c.move(1)
>>> c.move(0)
>>> print(c)
-------
-------
-------
-------
0------
01-----
```


In [64]:
grader.check("q2b")

q2b results: All test cases passed!

**2(c)** (8 pts) The `move` method should accept a single argument, `i`, and simulate the player whose turn it is dropping their marker in column `i`. If the move results in the player winning the game, the method returns `0` if the first player won, and `1` if the second player won. Otherwise it returns nothing. If an invalid column is passed in, the method should raise a `ValueError`.
```
>>> c = ConnectK(1, 3, 2)
ConnectK(rows=1, cols=3, k=2)
>>> c.move(0) 
>>> c.move(2)
>>> print(c)
0-1
>>> c.move(0)  # column zero is full
ValueError: ...
>>> c.move("A")  # invalid column
ValueError: ...
>>> c.move(1)  # player 0 wins
0
>>> print(c)
001
```

In [65]:
grader.check("q2c")

q2c results: All test cases passed!

**2(d)** (2 pts) The `possible_moves` property should return a list of all the possible moves for the current player. 
```
>>> c = ConnectK(1, 3, 2)
ConnectK(rows=1, cols=3, k=2)
>>> c.possible_moves
[0, 1, 2]
>>> c.move(1)
>>> print(c)
-0-
>>> c.possible_moves
[0, 2]
>>> c.move(2)
>>> print(c)
-01
>>> c.possible_moves
[0]
```

In [66]:
grader.check("q2d")

q2d results: All test cases passed!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [67]:
grader.check_all()

1a results: All test cases passed!

q1b results: All test cases passed!

q1c results: All test cases passed!

q1d results: All test cases passed!

q1e results: All test cases passed!

q1f results: All test cases passed!

q1g results: All test cases passed!

q1h results: All test cases passed!

q1i results: All test cases passed!

q1j results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q2c results: All test cases passed!

q2d results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload this .zip file to Gradescope for grading.

In [68]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)